In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

driver = webdriver.Chrome('D:/GitHub/SeleniumDriver/Chrome/chromedriver.exe')
driver.maximize_window()
driver.get("https://www.tdcc.com.tw/QStatWAR/inputframe.htm")
time.sleep(2)

#soup = BeautifulSoup(driver.page_source,'lxml')
#print(soup.prettify())

parentHandle = driver.window_handles[0]
parentHandle_title = driver.title
#print(parentHandle_title)

#<select name="Report" onchange="displayField(this)">
select = Select(driver.find_element_by_name('Report'))
#<option value="indw003">上櫃混藏保管有價證券週餘額表</option>
select.select_by_value('indw003')

#<input type="submit" value="查詢" name="queryButton">
confirm_button = driver.find_element_by_name('queryButton')

confirm_button.click()
time.sleep(5)
childHandle=driver.window_handles[1]
driver.switch_to.window(childHandle)


In [21]:
#code to extract data to excel
soup = BeautifulSoup(driver.page_source, 'lxml')
weeklyVol = soup.find("table", attrs={"class":"mt"})
weeklyVol_data = weeklyVol.tbody.find_all('tr')
print(weeklyVol_data)

#print(soup.prettify())
#for a in soup.find_all(class_='wu19', align='left'):
#    print(a.string)
#code_cols = soup.find_all(class_='wul9', align='left')
#for code_coln in code_cols:
#    print(code_coln.text)

# Get all the headings of list


TypeError: 'NoneType' object is not callable

In [4]:
driver.close()
driver.switch_to.window(parentHandle)
driver.close()